In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from bg_net import client
from bg_net.utils import get_most_recent
from bg_net.injection_efficiency.config import config
from bg_net.injection_efficiency import data_utils
from bg_net.archiver.indexer_utils import str_to_jst_datetime

In [ ]:
OBS_HER = "TM_EVR0_HER_INJ_EFF_BCM"
OBS_LER = "TM_EVR0_LER_INJ_EFF_BCM"
target_key = OBS_LER
# Set path to trained injection efficiency model
path_to_model = "/home/lukas/bg_net/examples/serving_model/" + target_key
# Get names of model inputs
input_features = data_utils.get_input_feature_names(target_key)

In [ ]:
model = tf.saved_model.load(get_most_recent(path_to_model))

In [ ]:
# The constraint for the baysian optimization search space in the format [{PV}, {middle}, {range around middle}]
optimization_space = np.array([
    ["CGLINJ_SEPTUM_ANG_R", 4.13, 6e-02],
    ["CGLINJ_SEPTUM_POS_R", 34, 6e-02],
    ["CGLINJ_INJECTION_YPOS", -0.00023, 1e-05],
    ["CGLINJ_INJECTION_YANG", -0.000231, 1e-05],
])
bunch_current = 0.610932

In [ ]:
# To use the model for inference we need to first define a point in the state space
state = [4.13, 34, -0.00023, -0.000231]
# This point is converted to a pandas dataframe with named columns and the bunch current is appended
df = pd.DataFrame(data=[np.append(state, bunch_current)],
              columns=np.append(optimization_space[:,0], 'LER_BUNCH_CURRENT'))

In [ ]:
# We can now do a prediction of the injection efficiency on our new point
serialized_example_batch = client.get_serialized_examples_from_df(df, target_key, input_features)
infer = model.signatures["serving_default"]
y_pred = infer(serialized_example_batch)['output_0'].numpy().squeeze()